# FAKE NEWS CLASSIFIER USING LSTM & RNN

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import tensorflow as tf

In [5]:
from keras.models import Sequential

In [7]:
from keras.layers import *

In [9]:
from keras.preprocessing.sequence import pad_sequences

In [11]:
from keras.preprocessing.text import one_hot

In [12]:
from keras.layers import LSTM

In [35]:
df=pd.read_csv("news.csv")

In [36]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [37]:
from sklearn.preprocessing import LabelEncoder

In [38]:
ll=LabelEncoder()

In [39]:
ll.fit_transform(df["label"])

array([0, 0, 1, ..., 0, 1, 1])

In [40]:
df["label"]=ll.fit_transform(df["label"])

In [41]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


In [42]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [43]:
df.dropna()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


In [44]:
df=df.drop(["Unnamed: 0"],axis=1)

In [45]:
df.columns

Index(['title', 'text', 'label'], dtype='object')

In [46]:
x=df.drop(["label"],axis=1)

In [49]:
y=df["label"]

In [50]:
import nltk

In [60]:
import re

# it is for regular expression 

In [52]:
from nltk.corpus import stopwords

In [54]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [59]:
message=x.copy()

# STOPWORDS & STEMMING 

In [75]:
from nltk.stem.porter import PorterStemmer

In [76]:
ps=PorterStemmer()

In [77]:
corpus=[]

for i in range (0,len(message)):
    review=re.sub("[^a-zA-Z]"," ",message["title"][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words("english")]
    
    review=" ".join(review)
    corpus.append(review)

In [78]:
corpus

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk',
 'three clinton iowa glimps fire elud hillari clinton campaign',
 'donald trump shockingli weak deleg game somehow got even wors',
 'strong solar storm tech risk today news oct video',
 'way america prepar world war',
 'trump take cruz lightli',
 'women lead differ',
 'shock michel obama hillari caught glamor date rape promot',
 'hillari clinton huge troubl america notic sick thing hidden pictur liberti writer news',
 'iran bill obama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope trump propos ban muslim',
 'episod sunday wire hail deplor special g

# ONE HOT CODEING REPRESENTATION

In [79]:
voc_size=10000

In [80]:
from keras.preprocessing.text import one_hot

In [81]:
one_hor=[one_hot(words,voc_size) for words in corpus]

In [82]:
one_hor

[[8528, 540, 9308],
 [8418, 4830, 7123, 9741, 6777, 1253, 4796, 5660, 4863, 1914, 6065],
 [6996, 8887, 9204, 7358, 1103],
 [5823, 7884, 7594, 9948, 8676, 8480, 396, 8659],
 [1847, 1414, 4825, 9281, 7928],
 [8108, 5298],
 [8319, 7278, 8418, 3215, 839, 2432],
 [5989, 4515, 3956],
 [4477, 3050, 4863, 193, 3558, 5701, 1034],
 [7948, 9026, 4489, 1414, 4677, 6093, 9418, 2107, 6312],
 [5725, 193, 164, 8552, 7203, 3451, 540, 193, 6477],
 [7306, 4863, 5874, 4190, 2330, 527, 6144, 189, 5026, 3029],
 [1796, 5874, 6034, 1522, 8696, 3621, 8340, 2153, 6065],
 [7261, 4682, 4572, 9963, 1735],
 [4863, 2421, 3100, 8678],
 [2367, 9644, 4700],
 [5390, 2149, 8035, 540, 7189, 4138, 8655, 926, 8615],
 [540, 193, 592, 4853, 4682, 2996, 5397, 7426, 4939, 3361, 3548, 9405, 8340],
 [7948, 6923, 8035, 2811],
 [2209, 7996, 6925, 7415, 1825, 7049, 4682],
 [5401, 9907, 4863, 9995, 5493, 6092],
 [7929, 2571, 600, 2898, 3544, 367, 2127, 2432, 6102],
 [540, 193, 4489, 6468, 8034, 6348, 6008],
 [1414, 9944, 5993, 3931, 

# Embedding Representation

In [83]:
from keras.preprocessing.sequence import pad_sequences

In [84]:
sent_length=20

In [89]:
embadding_data=pad_sequences(one_hor,padding="pre",maxlen=sent_length)

In [90]:
embadding_data

array([[   0,    0,    0, ..., 8528,  540, 9308],
       [   0,    0,    0, ..., 4863, 1914, 6065],
       [   0,    0,    0, ..., 9204, 7358, 1103],
       ...,
       [   0,    0,    0, ..., 3927, 4406, 7059],
       [   0,    0,    0, ..., 8622,  327, 6874],
       [   0,    0,    0, ..., 3194, 4863, 7928]])

In [91]:
embadding_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 8528,  540, 9308])

# model creation

In [98]:
embedding_vector_featureing=40

In [99]:
model=Sequential()

In [100]:
model.add(Embedding(voc_size,embedding_vector_featureing,input_length=sent_length))
model.add(LSTM(100)) # in this LSTM we given 1 layer of 100 neuron

model.add(Dense(1,activation="sigmoid"))


In [107]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [108]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            400000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [118]:
from keras.utils.vis_utils import plot_model

In [119]:
plot_model(model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [127]:
x_t=np.array(embadding_data)

In [128]:
y_t=np.array(y)

# splitting data

In [126]:
from sklearn.model_selection import train_test_split

In [129]:
x_train,x_test,y_train,y_test=train_test_split(x_t,y_t,random_state=1,test_size=0.2)

# fitting model & finding accuracy

In [132]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64,epochs=10)

Epoch 1/10
80/80 [==============================] - 7s 44ms/step - loss: 0.6167 - accuracy: 0.6535 - val_loss: 0.4748 - val_accuracy: 0.7719
Epoch 2/10
80/80 [==============================] - 3s 36ms/step - loss: 0.3565 - accuracy: 0.8483 - val_loss: 0.4519 - val_accuracy: 0.7940
Epoch 3/10
80/80 [==============================] - 3s 36ms/step - loss: 0.2401 - accuracy: 0.9069 - val_loss: 0.4845 - val_accuracy: 0.7940
Epoch 4/10
80/80 [==============================] - 3s 37ms/step - loss: 0.1685 - accuracy: 0.9386 - val_loss: 0.5609 - val_accuracy: 0.7885
Epoch 5/10
80/80 [==============================] - 3s 36ms/step - loss: 0.1268 - accuracy: 0.9556 - val_loss: 0.6336 - val_accuracy: 0.7719
Epoch 6/10
80/80 [==============================] - 3s 36ms/step - loss: 0.0944 - accuracy: 0.9649 - val_loss: 0.7670 - val_accuracy: 0.7585
Epoch 7/10
80/80 [==============================] - 3s 37ms/step - loss: 0.0709 - accuracy: 0.9742 - val_loss: 0.9830 - val_accuracy: 0.7609
Epoch 8/10
80

In [133]:
from sklearn.metrics import classification_report,accuracy_score

In [135]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [182]:
y_pred=model.predict(x_test)